# Noteebook to run Neuralangelo

NOTE: containers/pods for Colmap and neuralangelo are seperate as they have conflicting dependencies.


### Git Repo
Get latest code repo:

In [1]:
!git reset --hard HEAD
!git clean -xffd
!git pull

HEAD is now at f7d009f Update code for runpod secrets support
Already up to date.


### Slack Bot

In [5]:
import os
import sys

# Get the absolute path of project_root
project_root = os.path.abspath(os.path.join(os.path.dirname('__file__'), ".."))
# Add project_root to sys.path
sys.path.append(project_root)

from neovoxel_bot import neovoxel_bot
# Create Slack bot client:
bot = neovoxel_bot.NeoVoxelBot(channel='neovoxel-bot')

## Data:

1. Get the data.
2. Generate config.yaml file

In [ ]:
# Make data directory
import os

current_path = os.path.abspath(os.getcwd())
print(f'{current_path=}')
data_dir = os.path.abspath(os.path.join(current_path, os.pardir, 'datasets'))
print(f'{data_dir=}')

# Make dirs:
if os.path.exists(data_dir):
    print("[INFO] DATASETS DIRECTORY EXISTS")
else:
    print("[INFO] CREATING DATASETS DIRECTORY")
    os.makedirs(data_dir)


In [ ]:
# Download data:

# Get runpod_cmd from the colmap pod.
runpod_cmd = ''

#change directory to datasets directory and run runpod cmd. 
%pwd
%cd {data_dir}

!{runpod_cmd}

%cd {current_path}
%pwd


In [ ]:
# Generate config file:
    # data_path: absolute path to dataset
    # SEQUENCE: your custom name for the video sequence.
    # SCENE_TYPE: can be one of  {outdoor,indoor,object}.

DATA_PATH = '/' # copy the path the dataset was saved to. path in previous cell output.
SEQUENCE = ''
SCENE_TYPE = 'object'

!python3 projects/neuralangelo/scripts/generate_config.py --sequence_name {SEQUENCE} --data_dir {DATA_PATH} --scene_type {SCENE_TYPE}

# RUN NEURALANGELO:
    1. Populate variables
    2. Run Command

In [ ]:
EXPERIMENT=SEQUENCE
GROUP='example_group'
NAME='example_name'
CONFIG='/' #config path output from the above cell.
GPUS=1  # use >1 for multi-GPU training!

#Move control to right folder - neuralangelo
%pwd
%cd '/workspace/neuralangelo/'

### Without Weights and Biases

In [ ]:
# Without W&B:
bot.send_message(f'*NEURALANGELO*: Training _{SEQUENCE}_ started')
try:
    !torchrun --nproc_per_node={GPUS} train.py \
        --logdir=logs/{GROUP}/{NAME} \
        --config={CONFIG} \
        --show_pbar 
except Exception as e:
    bot.send_message(f'*NEURALANGELO*: error:_{e}_')
bot.send_message(f'*NEURALANGELO*: Training completed {SEQUENCE}')

### With Weights and Biases

In [ ]:
#How to get API KEY? ----> https://docs.wandb.ai/support/find_api_key/
bot.send_message(f'*NEURALANGELO*: Training _{SEQUENCE}_ started')
try:
    !wandb login <YOUR_API_KEY>
    !torchrun --nproc_per_node={GPUS} train.py \
        --logdir=logs/{GROUP}/{NAME} \
        --config={CONFIG} \
        --show_pbar \
        --wandb \
        --wandb_name={SEQUENCE}
except Exception as e:
    bot.send_message(f'*NEURALANGELO*: error:_{e}_')
bot.send_message(f'*NEURALANGELO*: Training completed {SEQUENCE}')

# EXTRACT MESH

Download trhe generated file and view it on a 3D viewer. 

In [ ]:
CHECKPOINT='' #.pt
OUTPUT_MESH='' #.ply
CONFIG='' #This is the config from the training logs. [logs/<group>/<name>/config.yaml]
RESOLUTION=2048 # options: 2048, 1024, 512, 256
BLOCK_RES=128
GPUS=1  # use >1 for multi-GPU mesh extraction


In [ ]:
!torchrun --nproc_per_node={GPUS} projects/neuralangelo/scripts/extract_mesh.py \
    --config={CONFIG} \
    --checkpoint={CHECKPOINT} \
    --output_file={OUTPUT_MESH} \
    --resolution={RESOLUTION} \
    --block_res={BLOCK_RES} \
    --textured